
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Community</a>
    </p>
</center>

# <center>Bedrock Tracing</center>

This guide demonstrates how to use Arize for monitoring and debugging your LLM using Traces and Spans. You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to send traces to Arize.

ℹ️ This notebook requires:
- An AWS account
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [ ]:
!pip install openinference-instrumentation-bedrock
!pip install opentelemetry-exporter-otlp
!pip install arize-otel
!pip install boto3

## Step 2: Get your API Keys

Copy the Arize `API_KEY` and `SPACE_ID` from your Space Settings page (shown below) to the variables in the cell below.

Follow this guide for setting up your [AWS credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html). You will need to enable model access for Bedrock [here](https://us-east-2.console.aws.amazon.com/bedrock/home#/modelaccess).

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [ ]:
import os
from getpass import getpass

SPACE_ID = globals().get("SPACE_ID") or getpass(
    "🔑 Enter your Arize Space ID: "
)
API_KEY = globals().get("API_KEY") or getpass("🔑 Enter your Arize API Key: ")

AWS_ACCESS_KEY_ID = globals().get("AWS_ACCESS_KEY_ID") or getpass(
    "🔑 Enter your AWS Access Key ID: "
)
AWS_SECRET_ACCESS_KEY = globals().get("AWS_SECRET_ACCESS_KEY") or getpass(
    "🔑 Enter your AWS Secret Access Key: "
)
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
# OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
#     "🔑 Enter your OpenAI API key: "
# )
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Step 3. Add our tracing code
We will be using the `arize-otel` package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [ ]:
# Import open-telemetry dependencies
from arize.otel import register

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="tracing-haiku-tutorial",
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.bedrock import BedrockInstrumentor

# Start the instrumentor for Bedrock
BedrockInstrumentor().instrument(tracer_provider=tracer_provider)

## Step 4: Run your LLM application
Let's test our app by asking to write a haiku. If you have difficulty invoking the model, you can change the modelId to the ARN of a model that you have access to ([see guide here](https://repost.aws/questions/QUEU82wbYVQk2oU4eNwyiong/bedrock-api-invocation-error-on-demand-throughput-isn-s-supported)).

The invocation parameters are also different for Bedrock vs. traditional LLM inference. Here's the Bedrock docs for [invoke_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html#BedrockRuntime.Client.invoke_model) vs. [converse](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html#converse).

In [ ]:
import boto3
import json

session = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)
client = session.client("bedrock-runtime", region_name="us-east-2")
model_id = ""

messages = [{"role": "user", "content": [{"text": "Hello, how are you?"}]}]

response = client.converse(
    messages=messages,
    modelId=model_id,
)
import pprint

pprint.pprint(response.get("output"))

This also works with `invoke_model`.

In [ ]:
prompt = b'{"prompt": "Human: Hello there, how are you? Assistant:"}'
response = client.invoke_model(modelId=model_id, body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body)

Here's an example with boto3 agents. You will need to set up an agent in the AWS console, make sure it's deployed, and then create an alias for it. Make sure the region_name is correct or else the agent will not work.

Example URL to access the console: https://us-east-2.console.aws.amazon.com/bedrock/home?region=us-east-2#agents

<img src="https://storage.googleapis.com/arize-assets/fixtures/amazon-bedrock-agents.png" width="700">

<img src="https://storage.googleapis.com/arize-assets/fixtures/amazon-bedrock-agents-setup.png" width="700">

In [ ]:
import time

runtime_client = boto3.client("bedrock-agent-runtime", region_name="us-east-2")
timestamp = int(time.time())
response = runtime_client.invoke_agent(
    agentId="",
    agentAliasId="",
    inputText="write me a haiku about a cat",
    sessionId=f"default-session1-{timestamp}",
    enableTrace=True,
)
completion = ""
for event in response.get("completion"):
    if "chunk" in event:
        completion += event["chunk"]["bytes"].decode()
    elif "trace" in event:
        print(event["trace"])
print(completion)

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`.